In [1]:
from data_preparation import *

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras import regularizers

2023-09-04 11:20:25.253454: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-04 11:20:25.254649: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 11:20:25.277729: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 11:20:25.278520: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 11:20:25.720502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
folder_list = extract_file_list()
folder_list

['flag',
 'resistance',
 'h&s_top',
 'vertical_run_up',
 'h&s_bottom',
 'flat_base',
 'double_top',
 'pennant',
 'horizontal_channel',
 'descending_channel',
 'support',
 'falling_wedge',
 'rising_wedge',
 'ascending_triangle',
 'downtrend',
 'ascending_channel',
 'double_bottom',
 'uptrend',
 'triangle']

# Model

In [3]:
def initialize_model_CNN(input_shape):
    model = Sequential()

    model.add(Masking(mask_value=-1, input_shape=input_shape))
    model.add(Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))),
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, activation='relu', kernel_size=3))
    
    model.add(Flatten())
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=3, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [4]:
patterns_to_test = ['rising_wedge', 'falling_wedge', 'double_top', 'double_bottom']
results_dict = {}

# Rising Wedge

In [5]:
X_real, y_real = real_data_pattern(patterns_to_test[0])
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[0])
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[0])

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [6]:
y_all = np.array(y_joined)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

2023-09-04 11:20:50.387661: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-04 11:20:50.387940: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
21/21 [==============================] - 1s 18ms/step - loss: 325938.2188 - mae: 108.2924 - val_loss: 26005.5254 - val_mae: 56.6517
Epoch 2/50
21/21 [==============================] - 0s 11ms/step - loss: 29460.0117 - mae: 49.0850 - val_loss: 4653.9912 - val_mae: 36.8474
Epoch 3/50
21/21 [==============================] - 0s 11ms/step - loss: 5039.4868 - mae: 35.9198 - val_loss: 3920.8623 - val_mae: 32.3265
Epoch 4/50
21/21 [==============================] - 0s 11ms/step - loss: 2730.2109 - mae: 32.7783 - val_loss: 3254.7761 - val_mae: 28.2228
Epoch 5/50
21/21 [==============================] - 0s 11ms/step - loss: 1897.6499 - mae: 26.7592 - val_loss: 3076.0120 - val_mae: 26.7313
Epoch 6/50
21/21 [==============================] - 0s 12ms/step - loss: 1612.1925 - mae: 24.6248 - val_loss: 2747.3074 - val_mae: 26.2839
Epoch 7/50
21/21 [==============================] - 0s 11ms/step - loss: 1347.1632 - mae: 22.0349 - val_loss: 2375.8059 - val_mae: 22.7083
Epoch 8/50
21/21 [====

In [8]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

results_dict['Rising Wedge'] = res[1]

11/11 [==============================] - 0s 5ms/step - loss: 2116.6658 - mae: 18.4724


# Falling wedge

In [9]:
X_real, y_real = real_data_pattern(patterns_to_test[1])
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[1])
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[1])

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [10]:
y_all = np.array(y_joined)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [11]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
14/14 [==============================] - 1s 22ms/step - loss: 63317.3320 - mae: 56.6953 - val_loss: 140806.8594 - val_mae: 82.3932
Epoch 2/50
14/14 [==============================] - 0s 13ms/step - loss: 27690.9414 - mae: 43.5016 - val_loss: 66239.6875 - val_mae: 60.2301
Epoch 3/50
14/14 [==============================] - 0s 12ms/step - loss: 6905.6484 - mae: 36.7239 - val_loss: 128072.1016 - val_mae: 66.7053
Epoch 4/50
14/14 [==============================] - 0s 12ms/step - loss: 2984.7866 - mae: 28.7385 - val_loss: 169433.5625 - val_mae: 57.6827
Epoch 5/50
14/14 [==============================] - 0s 12ms/step - loss: 2389.9331 - mae: 21.7121 - val_loss: 2313.8582 - val_mae: 23.7020
Epoch 6/50
14/14 [==============================] - 0s 13ms/step - loss: 2145.5708 - mae: 23.6300 - val_loss: 2501.7295 - val_mae: 24.7018
Epoch 7/50
14/14 [==============================] - 0s 12ms/step - loss: 1599.1871 - mae: 19.6224 - val_loss: 2994.3435 - val_mae: 24.2362
Epoch 8/50
14/14 [

In [12]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

results_dict['Falling Wedge'] = res[1]

8/8 [==============================] - 0s 5ms/step - loss: 525.2982 - mae: 10.1955


# Double Top

In [13]:
X_real, y_real = real_data_pattern(patterns_to_test[2])
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[2])
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[2])

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [14]:
y_all = np.array(y_joined)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [15]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
14/14 [==============================] - 1s 20ms/step - loss: 166212.5156 - mae: 78.6887 - val_loss: 13749.0742 - val_mae: 58.9153
Epoch 2/50
14/14 [==============================] - 0s 12ms/step - loss: 469690.0000 - mae: 83.2825 - val_loss: 11546.1562 - val_mae: 53.9562
Epoch 3/50
14/14 [==============================] - 0s 12ms/step - loss: 129711.9297 - mae: 69.3772 - val_loss: 10390.9277 - val_mae: 53.1010
Epoch 4/50
14/14 [==============================] - 0s 13ms/step - loss: 109403.5391 - mae: 64.2828 - val_loss: 10556.7334 - val_mae: 48.7397
Epoch 5/50
14/14 [==============================] - 0s 13ms/step - loss: 33722.0625 - mae: 54.4102 - val_loss: 9868.0869 - val_mae: 47.5284
Epoch 6/50
14/14 [==============================] - 0s 14ms/step - loss: 20747.0840 - mae: 51.5802 - val_loss: 9527.0869 - val_mae: 49.2343
Epoch 7/50
14/14 [==============================] - 0s 12ms/step - loss: 13034.0342 - mae: 47.9405 - val_loss: 8996.7109 - val_mae: 47.1937
Epoch 8/50
1

In [16]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

results_dict['Double Top'] = res[1]

8/8 [==============================] - 0s 5ms/step - loss: 1572.8999 - mae: 17.4309


# Double Bottom

In [17]:
X_real, y_real = real_data_pattern(patterns_to_test[3])
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[3])
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[3])

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [18]:
y_all = np.array(y_joined)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [19]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
13/13 [==============================] - 1s 22ms/step - loss: 10620.5312 - mae: 50.0081 - val_loss: 7972.4033 - val_mae: 44.1575
Epoch 2/50
13/13 [==============================] - 0s 13ms/step - loss: 8822.6143 - mae: 44.9618 - val_loss: 7307.5752 - val_mae: 40.0536
Epoch 3/50
13/13 [==============================] - 0s 13ms/step - loss: 8018.6045 - mae: 38.9236 - val_loss: 7216.6963 - val_mae: 39.7842
Epoch 4/50
13/13 [==============================] - 0s 12ms/step - loss: 8023.6958 - mae: 39.9505 - val_loss: 7822.5767 - val_mae: 44.1486
Epoch 5/50
13/13 [==============================] - 0s 12ms/step - loss: 7180.6646 - mae: 40.3660 - val_loss: 7344.7119 - val_mae: 42.0712
Epoch 6/50
13/13 [==============================] - 0s 14ms/step - loss: 6431.0996 - mae: 35.8973 - val_loss: 13158.5000 - val_mae: 49.3226
Epoch 7/50
13/13 [==============================] - 0s 14ms/step - loss: 6950.2710 - mae: 36.4115 - val_loss: 6747.4517 - val_mae: 38.6343
Epoch 8/50
13/13 [=======

In [20]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

results_dict['Double Bottom'] = res[1]

7/7 [==============================] - 0s 6ms/step - loss: 2583.7886 - mae: 24.0598


# Results

In [23]:
results_df = pd.DataFrame(results_dict, index=[1])
results_df

,Rising Wedge,Falling Wedge,Double Top,Double Bottom
1,18.472412,10.195467,17.430899,24.059771
